# Path planning and assembly process simulation 

## Path planning

* Collision checking (consider collisions)
* Trajectory checking (synchronization, consider speed and acceleration of moving objects) 

<img src="images/path_planning_00.png" width="600" />

# Collision checking

* Intricate positions (spatial assembly)
* Multiple robots working closely

<table>
<tr>
<td><img src="http://www.dfab.arch.ethz.ch/data/ProjectImages/02_Web/M/248/190617_248_robotic_setup_3x2_AX_WM.jpg" style="height: 300px" /></td>
<td><img src="http://www.dfab.arch.ethz.ch/data/ProjectImages/02_Web/M/188/160922_188_BuildupPrototype_MP_01_WM.jpg" style="height: 300px" /></td>
</tr>
</table>



# Trajectory checking

* Synchronisation
* Continuous processes

<table>
<tr>
<td><img src="images/path_planning_04.jpg" style="height: 300px" /></td>
<td><img src="http://www.dfab.arch.ethz.ch/web/images/content/GKR_Infrastructure_7.jpg" style="height: 300px" /></td>
</tr>
</table>


## MoveIt!

* ROS’ default motion planning library
* Works with different planners: OMPL (Open Motion Planning Library)
* Collision checking through FCL (Flexible Collision Library)
* Kinematics plugins (default KDL: Kinematics and Dynamics Library)
* Trajectory processing routine

<img src="https://moveit.ros.org/assets/images/moveit2_logo_black.png" width="300">


In [ ]:
docker compose up

## RViz in web browser


http://localhost:8080/vnc.html?resize=scale&autoconnect=true

<img src="images/moveit_ur5.jpg" width="700" />

<img src="images/moveit_rfl.jpg" width="700" />

<img src="images/moveit_ddad.jpg" width="700" />

In [ ]:
list services via roslibpy

In [ ]:
## Robot class in compas_fab

- model
- artist
- semantic
- client

In [16]:
import os
import compas
import compas_fab
from compas.robots import *
from compas_fab.robots import *
from compas_fab.backends import RosClient
from threejs_viewer import RobotArtist

urdf_filename = compas_fab.get('universal_robot/ur_description/urdf/ur5.urdf')
srdf_filename = compas_fab.get('universal_robot/ur5_moveit_config/config/ur5.srdf')

model = RobotModel.from_urdf_file(urdf_filename)
loader = LocalPackageMeshLoader(compas_fab.get('universal_robot'), 'ur_description')
model.load_geometry(loader)

semantics = RobotSemantics.from_srdf_file(srdf_filename, model)
artist = RobotArtist(model)

robot = Robot(model, artist=artist, semantics=semantics)
print("done")
    

done


In [10]:
robot.info()
print("")

The robot's name is 'ur5'.
The planning groups are: ['manipulator', 'endeffector']
The main planning group is 'manipulator'.
The end-effector's name is 'ee_link'.
The base link's name is 'base_link'
The base_frame is: Frame(Point(0.000, 0.000, 0.000), Vector(1.000, 0.000, 0.000), Vector(-0.000, 1.000, 0.000))
The robot's joints are:
	* 'shoulder_pan_joint' is of type 'revolute' and has limits [6.283, -6.283]
	* 'shoulder_lift_joint' is of type 'revolute' and has limits [6.283, -6.283]
	* 'elbow_joint' is of type 'revolute' and has limits [6.283, -6.283]
	* 'wrist_1_joint' is of type 'revolute' and has limits [6.283, -6.283]
	* 'wrist_2_joint' is of type 'revolute' and has limits [6.283, -6.283]
	* 'wrist_3_joint' is of type 'revolute' and has limits [6.283, -6.283]
The robot's links are:
['base_link', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'ee_link', 'base', 'tool0', 'world']



## Inverse kinematic

In [5]:
from compas.geometry import Frame
from compas_fab.backends import RosClient
from compas_fab.robots import Configuration

frame = Frame([0.3, 0.1, 0.5], [1, 0, 0], [0, 1, 0])
start_configuration = Configuration.from_revolute_values([0] * 6)
group = "manipulator" # or robot.main_group_name

with RosClient() as client:
    robot.client = client
    config = robot.inverse_kinematics(frame, start_configuration, group)
    print(config)




Twisted reactor is already running


Configuration((6.241, 4.320, 1.671, 0.292, -4.755, -3.142), (0, 0, 0, 0, 0, 0))


In [8]:
from threejs_viewer import ThreeJsViewer

robot.update(config)

geo = robot.draw_visual()
viewer = ThreeJsViewer()
viewer.show(geo)


Configuration((6.241, 4.320, 1.671, 0.292, -4.755, -3.142), (0, 0, 0, 0, 0, 0))


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(intensity=0.5, position=(0.0, 0.0, 1.0), quaterni…

In [14]:
## Joint Trajectory

from compas.robots import Joint
from compas_fab.robots import *

num_joints = 6
types = [Joint.REVOLUTE] * num_joints

p1 = JointTrajectoryPoint([1.571, 0, 0, 0.262, 0, 0], types)
p1.accelerations = [3.] * num_joints
p1.velocities = [3.] * num_joints
p1.time_from_start = Duration(2, 1293)

p2 = JointTrajectoryPoint([0.571, 0, 0, 0.262, 0, 0], types)
p1.accelerations = [3.] * num_joints
p1.velocities = [3.] * num_joints
p2.time_from_start = Duration(6, 0)

trj = JointTrajectory([p1, p2])
print(trj)


JointTrajectoryPoint((1.571, 0.000, 0.000, 0.262, 0.000, 0.000), (0, 0, 0, 0, 0, 0), (3.000, 3.000, 3.000, 3.000, 3.000, 3.000), (3.000, 3.000, 3.000, 3.000, 3.000, 3.000), (0.000, 0.000, 0.000, 0.000, 0.000, 0.000), Duration(2, 1293))


In [ ]:
# Cartesian path

* What is a cartesian path

In [18]:
from compas.geometry import Frame
from compas_fab.backends import RosClient
from compas_fab.backends.ros import MoveItErrorCodes
from compas_fab.robots import Configuration

frames = []
frames.append(Frame([0.3, 0.1, 0.5], [1, 0, 0], [0, 1, 0]))
frames.append(Frame([0.4, 0.3, 0.4], [0, 1, 0], [0, 0, 1]))
start_configuration = Configuration.from_revolute_values([-0.042, 4.295, -4.110, -3.327, 4.755, 0.])
group = "manipulator" # or robot.main_group_name

with RosClient() as client:
    robot.client = client
    trajectory = robot.plan_cartesian_motion(frames,
                                              start_configuration,
                                              max_step=0.01,
                                              group=group)

print(trajectory)    
print("Computed cartesian path with %d configurations, " % len(trajectory.points))
print("following %d%% of requested trajectory." % (trajectory.fraction * 100))
print("Executing this path at full speed would take approx. %.3f seconds." % trajectory.points[-1].time_from_start.seconds)

Twisted reactor is already running


Computed cartesian path with 1 configurations, 
following 0% of requested trajectory.
Executing this path at full speed would take approx. 0.000 seconds.
